# Script de Entrenamiento de Modelos    

En este código se encuentra el script base para el entrenamiento de modelos de clasificación para el script **penguins_size.csv**, por lo tanto el script está orientado para que se puedan hacer varios modelos. 

Hacemos el llamado de las librerías que necesitamos para el proceso

In [ ]:
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport 
from datetime import datetime

import os 
import joblib  
import pickle
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

##  Descriptivos Iniciales

Hacemos el reporte de los descriptivos generales del conjunto de datos para que el usuario pueda familiarizarse con los datos, iniciamos con la visualización del conjunto de datos

In [ ]:
print('Conjunto de Datos: ')
df_view = pd.read_csv("penguins_size.csv",sep=',')
print(df_view)

In [ ]:
print(df_view.info())

Hacemos la construcción de los descriptivos, y los visualizamos

In [ ]:
title_name = "Pinguins_class"
date_file_name = "ProfileReport_{}_{}.html".format(datetime.now().strftime('%Y-%m-%d'), title_name)

profile = ProfileReport(df_view, title='Profiling Report: ' + title_name, html={'style':{'full_width':True}})
profile.to_file(output_file=date_file_name)

In [ ]:
profile

##  Proceso de Entrenamiento de los Modelos

Para el proceso de entrenamiento de modelos nos orientaremos por la librería **scikit-learn** debido que esta librería es bastante completa para el entrenamiento de modelos de ML de manera fácil y sencilla

Limpieza de datos innecesarios

In [ ]:
df_view.dropna(inplace=True)
df_view = df_view[df_view['sex']!='.']
print(df_view)

Selección de muestra de entrenamiento y validación, para el flujo de entrenamiento de datos

In [ ]:
X = df_view.drop(columns=["species"])
y = df_view["species"]
text_cols = ["island", "sex"]
num_cols = [
    "culmen_length_mm", "culmen_depth_mm", "flipper_length_mm", "body_mass_g"
]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

preprocessor = ColumnTransformer(
    transformers=[
    ("text", OneHotEncoder(handle_unknown="ignore"), text_cols),
    ("num", StandardScaler(), num_cols)
])

###     Ejemplo de Entrenamiento de un Random Forest

In [ ]:
#   Definir el pipeline del modelo
modelRF = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(n_estimators=10, random_state=42))
])
#   Entrenar
modelRF.fit(X_train, y_train)
#   Evaluar
y_pred = modelRF.predict(X_test)
print("Precisión Modelo RF:", accuracy_score(y_test, y_pred))

# Guardar modelo en .pkl flotante
with open("modeloRF.pkl", "wb") as f:
    pickle.dump(modelRF, f)

###     Ejemplo de Entrenamiento de un SVM

In [ ]:
#   Definir el pipeline del modelo
modelSVM = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", SVC(kernel="linear", probability=True))
])
#   Entrenar
modelSVM.fit(X_train, y_train)
#   Evaluar
y_pred = modelSVM.predict(X_test)
print("Precisión Modelo SVM:", accuracy_score(y_test, y_pred))

# Guardar modelo en .pkl flotante
with open("modeloSVM.pkl", "wb") as f:
    pickle.dump(modelSVM, f)

##   Conectar con el Volumen Persistente

Ahora realizamos la conexión al volumen para guardar los modelos

In [ ]:
log_dir_docker = "/home/jovyan/work"
os.makedirs(log_dir_docker, exist_ok=True)

Guardamos los modelos

In [ ]:
joblib.dump(modelRF, os.path.join(log_dir_docker, "modeloRF.pkl"))

joblib.dump(modelSVM, os.path.join(log_dir_docker, "modeloSVM.pkl"))

Ahora se puede desarrollar otros modelos que recomendamos se encuentren dentro del repertorio que soporta **scikit-learn** 